In [ ]:
import numpy as np 

num_nurses = 4
num_days = 3
num_shifts = 3
shifts = ['AM', 'PM', 'Night']

# Simulated logistic regression probabilities 
# Shape: [nurse][day][shift]
prob_matrix = np.array([
    # Nurse 0
    [[0.9, 0.1, 0.2],  # Day 1
     [0.8, 0.2, 0.1],  # Day 2
     [0.9, 0.1, 0.2]], # Day 3

    # Nurse 1
    [[0.85, 0.1, 0.05], # Day 1
     [0.7, 0.15, 0.2], # Day 2
     [0.6, 0.25, 0.1]], # Day 3

    # Nurse 2
    [[0.1, 0.9, 0.2], # Day 1
     [0.2, 0.85, 0.1], # Day 2
     [0.1, 0.95, 0.2]], # Day 3 

    # Nurse 3
    [[0.2, 0.15, 0.85], # Day 1 
     [0.1, 0.2, 0.8], # Day 2 
     [0.1, 0.25, 0.85]], # Day 3 
]) 


In [18]:
from ortools.linear_solver import pywraplp

# OR-Tools Solver
solver = pywraplp.Solver.CreateSolver('SCIP')
x = {}

# Create binary decision vars: x[nurse, day, shift]
for n in range(num_nurses):
    for d in range(num_days):
        for s in range(num_shifts):
            x[n, d, s] = solver.IntVar(0, 1, f'x[{n},{d},{s}]')

# Constraint 1: Each shift per day is assigned to exactly 1 nurse
for d in range(num_days):
    for s in range(num_shifts):
        solver.Add(sum(x[n, d, s] for n in range(num_nurses)) == 1)

# Constraint 2: Each nurse works at most 1 shift per day
for n in range(num_nurses):
    for d in range(num_days):
        solver.Add(sum(x[n, d, s] for s in range(num_shifts)) <= 1)

# Objective: maximize total probability score
objective = solver.Objective()
for n in range(num_nurses):
    for d in range(num_days):
        for s in range(num_shifts):
            objective.SetCoefficient(x[n, d, s], prob_matrix[n][d][s])
objective.SetMaximization()

# Solve
status = solver.Solve()

# Output
if status == pywraplp.Solver.OPTIMAL:
    print(" Optimal assignment found:")
    for d in range(num_days):
        print(f"\n🗓 Day {d+1}")
        for s in range(num_shifts):
            for n in range(num_nurses):
                if x[n, d, s].solution_value() == 1:
                    print(f"   Shift {shifts[s]} → Nurse {n} (score: {prob_matrix[n][d][s]:.2f})")
    
else:
    print(" No optimal solution found.")

 Optimal assignment found:

🗓 Day 1
   Shift AM → Nurse 0 (score: 0.90)
   Shift PM → Nurse 2 (score: 0.90)
   Shift Night → Nurse 3 (score: 0.85)

🗓 Day 2
   Shift AM → Nurse 0 (score: 0.80)
   Shift PM → Nurse 2 (score: 0.85)
   Shift Night → Nurse 3 (score: 0.80)

🗓 Day 3
   Shift AM → Nurse 0 (score: 0.90)
   Shift PM → Nurse 2 (score: 0.95)
   Shift Night → Nurse 3 (score: 0.85)
